In [1]:
from sqlalchemy import create_engine
import pandas as pd
import sqlite3


In [10]:
# load data from database
engine = create_engine(
    'sqlite:///H:\\disaster-response-pipeline\\data\\disaster_record.db')

#read table and separate X and Y features
df = pd.read_sql_table('disaster_table', engine)

X = df.iloc[:, 0:4]

Y = df.iloc[:,5:40]

,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26211,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26214,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
